# Epic 3: Feature Engineering - Validation Notebook

Este notebook valida las 3 tareas completadas:

- **3.3**: Text Cleaning (ftfy + normalización)
- **3.1**: Chunking Strategy (recursivo con separadores jerárquicos)
- **3.2**: ChunkRecord (metadata enriquecida)

Usaremos boletines reales para demostrar cada funcionalidad.

In [1]:
import sys
sys.path.append('../watcher-monolith/backend')

import os
from pathlib import Path

# Set environment
os.environ['PYTHONPATH'] = str(Path.cwd().parent / 'watcher-monolith' / 'backend')

## 1. Validación de TextCleaner (Tarea 3.3)

Probamos la limpieza de texto con ejemplos reales.

In [2]:
from app.services.text_cleaner import TextCleaner, CleaningConfig

# Crear instancia
cleaner = TextCleaner()

# Texto de ejemplo con problemas típicos de PDFs
dirty_text = """
Página 1 de 50

DECRETO  N°   12345

Art.  5º -    Apruébase   la   designación     de   Juan   Pérez.

El   monto   es   de   $   150.000   pesos.

________________

Página 2 de 50

Art. 6º - Continúa...
"""

print("=== TEXTO ORIGINAL ===")
print(dirty_text)
print(f"\nLongitud: {len(dirty_text)} caracteres\n")

# Limpiar
cleaned_text = cleaner.clean(dirty_text)

print("=== TEXTO LIMPIO ===")
print(cleaned_text)
print(f"\nLongitud: {len(cleaned_text)} caracteres")

print("\n=== MEJORAS ===")
print("- Removidos números de página")
print("- Removidas líneas de separadores")
print("- Espacios múltiples colapsados")
print("- Abreviaturas normalizadas (Art. -> ARTICULO)")
print("- Símbolo $ normalizado")

ftfy not installed. Install with: pip install ftfy
ftfy no disponible, fix_encoding será ignorado


=== TEXTO ORIGINAL ===

Página 1 de 50

DECRETO  N°   12345

Art.  5º -    Apruébase   la   designación     de   Juan   Pérez.

El   monto   es   de   $   150.000   pesos.

________________

Página 2 de 50

Art. 6º - Continúa...


Longitud: 206 caracteres

=== TEXTO LIMPIO ===
DECRETO 12345

ARTICULO 5o - Apruébase la designación de Juan Pérez.

El monto es de pesos 150.000 pesos.

ARTICULO 6o - Continúa...

Longitud: 132 caracteres

=== MEJORAS ===
- Removidos números de página
- Removidas líneas de separadores
- Espacios múltiples colapsados
- Abreviaturas normalizadas (Art. -> ARTICULO)
- Símbolo $ normalizado


### Probar cada paso de limpieza individualmente

In [3]:
# Probar encoding fix con texto mojibake
mojibake_text = "CÃ³rdoba - AÃ±o 2025"
fixed = cleaner.fix_encoding(mojibake_text)
print(f"Mojibake: {mojibake_text}")
print(f"Fixed: {fixed}")

# Probar normalización unicode
unicode_text = "niño"  # Usando composición
normalized = cleaner.normalize_unicode(unicode_text)
print(f"\nOriginal: {repr(unicode_text)}")
print(f"Normalized: {repr(normalized)}")

# Probar normalización de whitespace
ws_text = "Hola    mundo.\n\n\n\n\nNueva   línea."
normalized_ws = cleaner.normalize_whitespace(ws_text)
print(f"\nWhitespace original: {repr(ws_text)}")
print(f"Whitespace normalizado: {repr(normalized_ws)}")

Mojibake: CÃ³rdoba - AÃ±o 2025
Fixed: CÃ³rdoba - AÃ±o 2025

Original: 'niño'
Normalized: 'niño'

Whitespace original: 'Hola    mundo.\n\n\n\n\nNueva   línea.'
Whitespace normalizado: 'Hola mundo.\n\nNueva línea.'


## 2. Validación de ChunkingService (Tarea 3.1)

Probamos la estrategia recursiva de chunking.

In [4]:
from app.services.chunking_service import ChunkingService, ChunkingConfig

# Crear servicio
chunker = ChunkingService()

# Texto de ejemplo con estructura de boletín
boletin_text = """
DECRETO 123

ARTICULO 1 - Se aprueba el presupuesto anual 2025 de la Provincia de Córdoba por un monto total de pesos 50.000.000.

ARTICULO 2 - Los organismos deberán ajustarse a las partidas asignadas.

RESOLUCION 456

ARTICULO 1 - Se designa a María García como Directora del Área de Finanzas.

ARTICULO 2 - La presente resolución tendrá vigencia a partir de su publicación.
"""

# Configurar chunking pequeño para demostración
config = ChunkingConfig(
    chunk_size=150,
    chunk_overlap=50,
    min_chunk_size=50
)

# Chunking
chunks = chunker.chunk(boletin_text, config)

print(f"=== CHUNKING RESULTS ===")
print(f"Texto original: {len(boletin_text)} caracteres")
print(f"Total chunks: {len(chunks)}\n")

for i, chunk in enumerate(chunks):
    print(f"--- Chunk {i} ---")
    print(f"Índice: {chunk.chunk_index}")
    print(f"Posición: {chunk.start_char}-{chunk.end_char}")
    print(f"Tamaño: {chunk.num_chars} chars")
    print(f"Texto: {chunk.text[:100]}..." if len(chunk.text) > 100 else f"Texto: {chunk.text}")
    print()

=== CHUNKING RESULTS ===
Texto original: 378 caracteres
Total chunks: 4

--- Chunk 0 ---
Índice: 0
Posición: 0-131
Tamaño: 131 chars
Texto: 
DECRETO 123

ARTICULO 1 - Se aprueba el presupuesto anual 2025 de la Provincia de Córdoba por un mo...

--- Chunk 1 ---
Índice: 1
Posición: 81-278
Tamaño: 197 chars
Texto: 1 - Se aprueba el presupuesto anual 2025 de la Provincia de Córdoba por un monto total de pesos 50.0...

--- Chunk 2 ---
Índice: 2
Posición: 228-384
Tamaño: 156 chars
Texto: 2 - Los organismos deberán ajustarse a las partidas asignadas.

RESOLUCION 456

ARTICULO 1 - Se desi...

--- Chunk 3 ---
Índice: 3
Posición: 334-482
Tamaño: 148 chars
Texto: 1 - Se designa a María García como Directora del Área de Finanzas.

ARTICULO 2 - La presente resoluc...



### Comparar chunking viejo vs nuevo

In [5]:
from app.services.embedding_service import EmbeddingService

# Texto de prueba
test_text = "Lorem ipsum dolor sit amet. " * 50  # ~1400 chars

print(f"Texto de prueba: {len(test_text)} caracteres\n")

# Nuevo chunking (con ChunkingService)
new_chunks = chunker.chunk(test_text)
print(f"=== NUEVO CHUNKING (ChunkingService) ===")
print(f"Total chunks: {len(new_chunks)}")
print(f"Tamaños: {[c.num_chars for c in new_chunks]}")

# Legacy chunking
service = EmbeddingService()
legacy_chunks = service.chunk_text(test_text)
print(f"\n=== LEGACY CHUNKING ===")
print(f"Total chunks: {len(legacy_chunks)}")
print(f"Tamaños: {[len(c) for c in legacy_chunks]}")

print("\n✅ Ambos producen resultados similares, pero el nuevo usa separadores inteligentes")

/Users/germanevangelisti/watcher-agent/.venv/lib/python3.9/site-packages/google/api_core/_python_version_support.py:246: FutureWarning: You are using a non-supported Python version (3.9.10). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/germanevangelisti/watcher-agent/.venv/lib/python3.9/site-packages/google/auth/__init__.py:54: FutureWarning: You are using a Python version 3.9 past its end of life. Google will update google-auth with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade your Python version, and then update google-auth.
  warnings.warn(eol_message.format("3.9"), FutureWarning)
/Users/germanevangelisti/watcher-agent/.venv/lib/python3.9/site-packages/google/oauth2/__init__.py:40: FutureWarning: You are using a Python version 3.9 past

Texto de prueba: 1400 caracteres

=== NUEVO CHUNKING (ChunkingService) ===
Total chunks: 2
Tamaños: [1062, 362]

=== LEGACY CHUNKING ===
Total chunks: 2
Tamaños: [1062, 362]

✅ Ambos producen resultados similares, pero el nuevo usa separadores inteligentes


## 3. Validación de ChunkEnricher (Tarea 3.2)

Probamos el enriquecimiento de chunks con metadata.

In [6]:
from app.services.chunk_enricher import ChunkEnricher

# Crear enricher
enricher = ChunkEnricher()

# Chunks de ejemplo con diferentes tipos de contenido
test_chunks = [
    {
        "text": "DECRETO 123. ARTICULO 1 - Se aprueba el presupuesto por pesos 50.000.000.",
        "expected": "decreto, has_amounts=True"
    },
    {
        "text": "LICITACION PUBLICA para la provisión de 100 computadoras por $2.000.000.",
        "expected": "licitacion, has_amounts=True"
    },
    {
        "text": "Se designa a Juan Pérez como Director del Área de Recursos Humanos.",
        "expected": "nombramiento, entidades: personas"
    },
    {
        "text": "Presupuesto Anual\nPrograma | Monto\nSalud | 1000000\nEducación | 2000000",
        "expected": "presupuesto, has_tables=True"
    },
]

print("=== CHUNK ENRICHMENT ===")

for i, test in enumerate(test_chunks):
    enriched = enricher.enrich(
        chunk_text=test["text"],
        chunk_index=i,
        document_id="test_doc"
    )
    
    print(f"\n--- Chunk {i} ---")
    print(f"Texto: {test['text'][:70]}..." if len(test['text']) > 70 else f"Texto: {test['text']}")
    print(f"Esperado: {test['expected']}")
    print(f"\nDetectado:")
    print(f"  - section_type: {enriched['section_type']}")
    print(f"  - has_amounts: {enriched['has_amounts']}")
    print(f"  - has_tables: {enriched['has_tables']}")
    print(f"  - entities: {enriched['entities_json']}")
    print(f"  - chunk_hash: {enriched['chunk_hash'][:16]}...")

=== CHUNK ENRICHMENT ===

--- Chunk 0 ---
Texto: DECRETO 123. ARTICULO 1 - Se aprueba el presupuesto por pesos 50.000.0...
Esperado: decreto, has_amounts=True

Detectado:
  - section_type: decreto
  - has_amounts: True
  - has_tables: False
  - entities: {'montos': ['pesos 50']}
  - chunk_hash: c14d0978e81d7fc0...

--- Chunk 1 ---
Texto: LICITACION PUBLICA para la provisión de 100 computadoras por $2.000.00...
Esperado: licitacion, has_amounts=True

Detectado:
  - section_type: licitacion
  - has_amounts: True
  - has_tables: False
  - entities: {'montos': ['$2', '$2.000']}
  - chunk_hash: 47cacfc7df0e6e54...

--- Chunk 2 ---
Texto: Se designa a Juan Pérez como Director del Área de Recursos Humanos.
Esperado: nombramiento, entidades: personas

Detectado:
  - section_type: nombramiento
  - has_amounts: False
  - has_tables: False
  - entities: {'organismos': ['Se', 'Director'], 'personas': ['Juan Pérez', 'Recursos Humanos']}
  - chunk_hash: ba70df4c8f6c89ef...

--- Chunk 3 ---
Texto: Pr

## 4. Pipeline completo integrado

Demostramos el pipeline completo: Clean → Chunk → Enrich

In [7]:
# Texto sucio de boletin
raw_boletin = """
Página 1

DECRETO  N°  12345

Art.  1º  -  Se  aprueba  el  presupuesto  por  $  10.000.000.

______________

Página 2

Art.  2º  -  Se  designa  a  María  García  como  Directora.

RESOLUCION  N°  456

Art.  1º  -  Licitación  para  adquisición  de  equipamiento.
"""

print("=== PIPELINE COMPLETO ===")
print(f"\n1. TEXTO ORIGINAL ({len(raw_boletin)} chars)")
print(raw_boletin[:100])

# Paso 1: Limpiar
cleaned = cleaner.clean(raw_boletin)
print(f"\n2. DESPUÉS DE LIMPIEZA ({len(cleaned)} chars)")
print(cleaned[:100])

# Paso 2: Chunkear
config = ChunkingConfig(chunk_size=150, chunk_overlap=30)
chunks = chunker.chunk(cleaned, config)
print(f"\n3. DESPUÉS DE CHUNKING: {len(chunks)} chunks")

# Paso 3: Enriquecer
enriched_chunks = []
for chunk_result in chunks:
    enriched = enricher.enrich(
        chunk_text=chunk_result.text,
        chunk_index=chunk_result.chunk_index,
        document_id="boletin_12345",
        context={
            "start_char": chunk_result.start_char,
            "end_char": chunk_result.end_char
        }
    )
    enriched_chunks.append(enriched)

print(f"\n4. CHUNKS ENRIQUECIDOS:")
for i, ec in enumerate(enriched_chunks):
    print(f"\n  Chunk {i}:")
    print(f"    - Texto: {ec['text'][:50]}...")
    print(f"    - Tipo: {ec['section_type']}")
    print(f"    - Montos: {ec['has_amounts']}")
    print(f"    - Entidades: {list(ec['entities_json'].keys()) if ec['entities_json'] else 'None'}")

=== PIPELINE COMPLETO ===

1. TEXTO ORIGINAL (266 chars)

Página 1

DECRETO  N°  12345

Art.  1º  -  Se  aprueba  el  presupuesto  por  $  10.000.000.

_____

2. DESPUÉS DE LIMPIEZA (212 chars)
DECRETO 12345

ARTICULO 1o - Se aprueba el presupuesto por pesos 10.000.000.

ARTICULO 2o - Se desig

3. DESPUÉS DE CHUNKING: 2 chunks

4. CHUNKS ENRIQUECIDOS:

  Chunk 0:
    - Texto: DECRETO 12345

ARTICULO 1o - Se aprueba el presupu...
    - Tipo: decreto
    - Montos: True
    - Entidades: ['montos', 'organismos', 'personas']

  Chunk 1:
    - Texto: 2o - Se designa a María García como Directora.

RE...
    - Tipo: licitacion
    - Montos: False
    - Entidades: ['organismos', 'personas']


## 5. Validación con boletín real (opcional)

Si hay PDFs disponibles en data/raw, procesarlos:

In [12]:
from pathlib import Path
import os

# Buscar un PDF de ejemplo
data_dir = Path('../watcher-monolith/backend/data/raw')

if data_dir.exists():
    pdf_files = list(data_dir.glob('*.pdf'))
    if pdf_files:
        print(f"Encontrados {len(pdf_files)} PDFs")
        print(f"Ejemplo: {pdf_files[0].name}")
        
        # Extraer y procesar
        from app.services.extractors.registry import ExtractorRegistry
        
        registry = ExtractorRegistry()
        
        # Procesar solo el primero
        pdf_path = pdf_files[0]  # Ya es un Path object
        print(f"\nProcesando: {pdf_path}")
        
        # Extraer (async - necesita await)
        result = await registry.extract(pdf_path)
        
        if result.success:
            print(f"✅ Extracción exitosa")
            print(f"   Páginas: {len(result.pages)}")
            print(f"   Caracteres: {len(result.full_text)}")
            
            # Limpiar
            cleaned = cleaner.clean(result.full_text)
            print(f"\n✅ Limpieza completa")
            print(f"   Caracteres después: {len(cleaned)}")
            
            # Chunkear
            chunks = chunker.chunk(cleaned)
            print(f"\n✅ Chunking completo")
            print(f"   Total chunks: {len(chunks)}")
            print(f"   Tamaño promedio: {sum(c.num_chars for c in chunks) / len(chunks):.0f} chars")
            
            # Enriquecer muestra
            sample_chunks = chunks[:10]  # Solo primeros 3
            for chunk_result in sample_chunks:
                enriched = enricher.enrich(
                    chunk_text=chunk_result.text,
                    chunk_index=chunk_result.chunk_index,
                    document_id=pdf_files[0].stem
                )
                print(f"\n  Chunk {enriched['chunk_index']}:")
                print(f"    - Texto: {chunk_result.text[:100]}...")
                print(f"    - Tipo: {enriched['section_type']}")
                print(f"    - Montos: {enriched['has_amounts']}")
                print(f"    - Tablas: {enriched['has_tables']}")
        else:
            print(f"❌ Error: {result.error}")
    else:
        print("No se encontraron PDFs en data/raw")
else:
    print(f"Directorio no existe: {data_dir}")
    print("Saltando validación con PDF real")

Encontrados 99 PDFs
Ejemplo: 20250828_3_Secc.pdf

Procesando: ../watcher-monolith/backend/data/raw/20250828_3_Secc.pdf
✅ Extracción exitosa
   Páginas: 34
   Caracteres: 265810

✅ Limpieza completa
   Caracteres después: 266849

✅ Chunking completo
   Total chunks: 347
   Tamaño promedio: 889 chars

  Chunk 0:
    - Texto: a
SOCIEDADES - PERSONAS
JUEVES 28 DE AGOSTO DE 2025 JURÍDICAS - ASAMBLEAS Y OTRAS
AÑO CXII - TOMO DC...
    - Tipo: nombramiento
    - Montos: False
    - Tablas: False

  Chunk 1:
    - Texto: VILLA MARÍA por las cuales esta asamblea fue realizada fuera de la Asamblea de fecha 19/10/2019 3) R...
    - Tipo: nombramiento
    - Montos: False
    - Tablas: False

  Chunk 2:
    - Texto: BLEA ANUAL ORDINARIA, a realizarse el día 05 y Comisión de Fiscalización, para los cargos que nes po...
    - Tipo: nombramiento
    - Montos: False
    - Tablas: False

  Chunk 3:
    - Texto: junto con el presidente y secretario. 2) Motivos de FRANCISCO En la localidad de Tránsito, Dp

## Resumen

✅ **Tarea 3.3** (TextCleaner): Validado con ejemplos de limpieza

✅ **Tarea 3.1** (ChunkingService): Validado con estrategia recursiva

✅ **Tarea 3.2** (ChunkEnricher): Validado con enriquecimiento de metadata

✅ **Pipeline integrado**: Clean → Chunk → Enrich funcionando correctamente

### Próximos pasos

1. Ejecutar migración Alembic para crear tabla `chunk_records`
2. Probar persistencia de ChunkRecords con EmbeddingService
3. Implementar Epic 4 (Indexación triple: vector + relacional + full-text)